After the first sweep, we thing that the models using the "invented" loss might have some results.
Let's visualize them:

In [1]:
from models import LSTMModel , LSTMModel_seeOnce
from dataset import ImageCaptionDataset

In [2]:
import pickle
from torch.utils.data import DataLoader
import wandb
import yaml
from typing import Dict
import gensim.downloader as api
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import torch
import torch.nn as nn

from tqdm import tqdm
from icecream import ic

import numpy as np

import os

device = "mps" #r u running cuda my boy? or mps? :D

#load datasets
with open('/Users/josepsmachine/Documents/UNI/DL/dlnn-project_ia-group_10/dataset/train_dataset.pkl', 'rb') as inp:
    train_dataset = pickle.load(inp)
with open('/Users/josepsmachine/Documents/UNI/DL/dlnn-project_ia-group_10/dataset/val_dataset.pkl', 'rb') as inp:
    val_dataset = pickle.load(inp)
#with open('/Users/josepsmachine/Documents/UNI/DL/dlnn-project_ia-group_10/dataset/debug_dataset.pkl', 'rb') as inp:
#    debug_dataset = pickle.load(inp)

#create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)
#debug_dataloader = DataLoader(debug_dataset, batch_size=1, shuffle=False)

#load hyperparamaters to do grid search on
#setup wandb stuff
with open('hyperparams.yaml', 'r') as stream:
    try:
        sweep_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


#load word2vec pretrained embedding layer
word2vec_emb = api.load('word2vec-google-news-300')
word2vec_emb = torch.FloatTensor(word2vec_emb.vectors)

#import vocabulary to word2vec indexes that we know
with open('vocabidx2word2vecidx.pkl', 'rb') as inp:
    vocabidx2word2vecidx = pickle.load(inp)

with open('vocabulary.pkl', 'rb') as inp:
    vocabulary = pickle.load(inp)

word2vec_emb = word2vec_emb[vocabidx2word2vecidx]

word2vec_emb = nn.Embedding.from_pretrained(word2vec_emb)
word2vec_emb.requires_grad_ = False #freeze word2vec embeddding layer


/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
emb_layer = nn.Embedding(num_embeddings=word2vec_emb.weight.shape[0], embedding_dim=300)

In [34]:
model = LSTMModel(input_dim=512,embedding_layer=emb_layer,hidden_dim=300,n_layers=1)

Load state from vm trained:

In [36]:
model.load_state_dict(torch.load("LSTM_residuals_v7sk01wn.pt",map_location=torch.device('cpu')))

<All keys matched successfully>

In [19]:
model.eval()

LSTMModel(
  (imf2lstm_h): Linear(in_features=512, out_features=300, bias=True)
  (imf2lstm_c): Linear(in_features=512, out_features=300, bias=True)
  (embedding): Embedding(8426, 300)
  (lstm): LSTM(300, 300, batch_first=True)
  (LM_FC): Linear(in_features=300, out_features=8426, bias=True)
  (softmax): Softmax(dim=1)
)

Let's get a sample from the validation dataloader:

In [20]:
X1,X2,caption = next(iter(val_dataloader))

In [21]:
caption

('bos man and woman kissing eos',)

In [22]:
X2.shape

torch.Size([1, 35])

In [28]:
model.embedding(torch.tensor([[1]])).shape

torch.Size([1, 1, 300])

In [33]:
out, _, _ = model(X1,X2,train=False)

TypeError: '_IncompatibleKeys' object is not callable

In [30]:
out.shape

torch.Size([1, 40, 8426])

In [37]:
sntc = ""
for tok in out[0]:
    #sample one token for each distrib
    step = torch.multinomial(tok, 1)
    sntc += " " + vocabulary[step]

print(caption)
print(sntc)

('bos man and woman kissing eos',)
 haircut hell packed opposition filming too six tightropes riverboat tether passageway contestant cycler along stunning jeep holding panda reflection ballerina firefighters circuit oncoming distribute narrow oddly dirtbikers laundry chew vegetables overflowing refrigerator upfront grapple annoyed swimsuites noticable chewing discovers when
